In [1]:
import numpy as np
import cv2
import ipywidgets
from IPython.display import display


# turn numpy array into a jpeg
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])



In [2]:

def gstreamer_pipeline(
    capture_width=640,
    capture_height=480,
    display_width=640,
    display_height=480,
    framerate=30,
    flip_method=0,
):
    return (
        "nvarguscamerasrc ! "
        "video/x-raw(memory:NVMM), "
        "width=(int)%d, height=(int)%d, "
        "format=(string)NV12, framerate=(fraction)%d/1 ! "
        "nvvidconv flip-method=%d ! "
        "video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! "
        "videoconvert ! "
        "video/x-raw, format=(string)BGR ! appsink max-buffers=1 drop=True"
        
        # "appsink max-buffers=1 drop=True" without this we have ahorrible lagging
        
        % (
            capture_width,
            capture_height,
            framerate,
            flip_method,
            display_width,
            display_height,
        )
    )


face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


In [3]:

image_widget = ipywidgets.Image(format='jpeg')
image_widget2 = ipywidgets.Image(format='jpeg')

display(ipywidgets.HBox([image_widget,image_widget2]))

In [ ]:
try:
    cap = cv2.VideoCapture("/dev/video1") #USB Camera
    cap2 = cv2.VideoCapture(gstreamer_pipeline(flip_method=2), cv2.CAP_GSTREAMER) # CSI Camera
    while 1:
        ret_val, img = cap.read()
        ret_val, img2 = cap2.read()
       
        faces1 = face_cascade.detectMultiScale(img, 1.7, 4)
        faces2 = face_cascade.detectMultiScale(img2, 1.7, 4)
        
        for (x, y, w, h) in faces1:
            cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
       
        for (x, y, w, h) in faces2:
            cv2.rectangle(img2, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        image_widget.value = bgr8_to_jpeg(img)      
        image_widget2.value = bgr8_to_jpeg(img2)

except KeyboardInterrupt:
    print("stop")
    cap.release()
    cap2.release()